# E-Bay Scraper
---
Webscrapper for E-bay listing history.
---
### Dependencies

In [1]:
from bs4 import BeautifulSoup
import requests
from splinter import Browser
from os.path import basename
from pprint import pprint
import time
from datetime import datetime as dt
import pandas as pd
import numpy as np

In [2]:
# Splinter set up
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

---
### Extraction
- Exploratory scrape of data off of E-bay

In [3]:
# Direct browser to E-bay to prepare for scraping

# sold_base_url = 'https://www.ebay.com/'
# scrape_target = 'funko+pop'
# back_sold_url = '&_sacat=0&LH_Sold=1&rt=nc&rt=nc&rt=nc&rt=nc&rt=nc&_ipg=200&rt=nc&_pgn=1'
# url = sold_base_url + scrape_target + back_sold_url
# browser.visit(url)





In [3]:
# Visit URL
base_url = 'https://www.ebay.com/'
browser.visit(base_url)

In [4]:
# Fill the search bar and click
browser.fill('_nkw', 'funko+pop')
browser.find_by_id('gh-btn').click()

In [5]:
# Specify search for sold items - found by counting inputs
browser.find_by_tag('input')[45].check()


In [6]:
# container for scrapes
sold_items = []

In [9]:
# Get html from page
html = browser.html
bs = BeautifulSoup(html, 'html.parser')

# Focus on 'listing' content
page_items = bs.findAll('div', {'class':'s-item__wrapper clearfix'})

# Inspect html
print(len(page_items))
print(page_items[0])


50
<div class="s-item__wrapper clearfix"><div class="s-item__image-section"><div class="s-item__image"><a _sp="p2351460.m1686.l7400" aria-hidden="true" data-track='{"eventFamily":"LST","eventAction":"ACTN","actionKind":"NAVSRC","actionKinds":["NAVSRC"],"operationId":"2351460","flushImmediately":false,"eventProperty":{"moduledtl":"mi:1686|iid:1|li:7400|luid:1|scen:Listings","parentrq":"8b3a1c381680ad4ebc190b27ffaf7872","pageci":"a175f507-2190-11e9-a2ae-74dbd180e494"}}' href="https://www.ebay.com/itm/Funko-Pop-Movies-114-THE-WOLF-MAN-Universal-Monsters-Vaulted-Box-damage/273670180469?hash=item3fb8038e75:g:i-EAAOSwvdFcSO~L" tabindex="-1"><div class="s-item__image-wrapper"><div class="s-item__image-helper"></div><img alt="Funko Pop! Movies #114 THE WOLF MAN Universal Monsters Vaulted Box damage" class="s-item__image-img" data-atftimer="1548524068555" onload='SITE_SPEED.ATF_TIMER.measure(this); if (performance &amp;&amp; performance.mark) { performance.mark("first-meaningful-paint"); }' src

In [10]:
# Check html for a item sold under list price
print(page_items[1])

<div class="s-item__wrapper clearfix"><div class="s-item__image-section"><div class="s-item__image"><a _sp="p2351460.m1686.l7400" aria-hidden="true" data-track='{"eventFamily":"LST","eventAction":"ACTN","actionKind":"NAVSRC","actionKinds":["NAVSRC"],"operationId":"2351460","flushImmediately":false,"eventProperty":{"moduledtl":"mi:1686|iid:2|li:7400|luid:1|scen:Listings","parentrq":"8b3a1c381680ad4ebc190b27ffaf7872","pageci":"a175f507-2190-11e9-a2ae-74dbd180e494"}}' href="https://www.ebay.com/itm/OOB-Lot-of-2-Han-Solo-Chewbacca-Funko-POP-Star-Wars-vinyl-figures/233093300602?hash=item364571817a:g:ojgAAOSwEvZcO8oA" tabindex="-1"><div class="s-item__image-wrapper"><div class="s-item__image-helper"></div><img alt="OOB Lot of 2 Han Solo Chewbacca Funko POP Star Wars vinyl figures" class="s-item__image-img" data-atftimer="1548524068829" onload="SITE_SPEED.ATF_TIMER.measure(this)" src="https://i.ebayimg.com/thumbs/images/g/ojgAAOSwEvZcO8oA/s-l225.jpg"/></div></a></div></div><div class="s-item_

In [16]:
links = browser.find_by_tag('a')
print(len(links))
print(links[0].html)

420
Skip to main content


In [42]:
html = browser.html
bs = BeautifulSoup(html, 'html.parser')
bs_links = bs.findAll('a')
print(len(bs_links))
print(bs_links[0])

417
<a class="gh-acc-a" href="#mainContent" id="gh-hdn-stm">Skip to main content</a>


In [43]:
counter = 0
for bs_link in bs_links:
    if bs_link.text.find('10') != -1:
        print(counter)
        print(bs_link)
    counter = counter + 1

124
<a _sp="p2351460.m4118.c4" class="cbx x-refine__multi-select-link" data-track='{"eventFamily":"LST","eventAction":"ACTN","actionKind":"NAVSRC","actionKinds":["NAVSRC"],"operationId":"2351460","flushImmediately":false,"eventProperty":{"moduledtl":"mi%3A4118%7Ciid%3A1%7Cselected%3A0%7Cglobal%3A0%7CaspectURLParam%3ARarity%7Cli%3A8173%7Cluid%3A43%7C","pageci":"e05bb510-2199-11e9-adda-74dbd180a003","parentrq":"8b76b3ce1680aa66b659e893ffb540da"}}' href="https://www.ebay.com/sch/i.html?_from=R40&amp;_nkw=funko%2Bpop&amp;_sacat=0&amp;LH_Sold=1&amp;LH_Complete=1&amp;rt=nc&amp;Rarity=Vaulted%252FRetired&amp;_dcat=149372"><input aria-hidden="true" aria-label="Vaulted/Retired" autocomplete="off" class="cbx x-refine__multi-select-checkbox " id="w_1548528039768_cbx" role="presentation" tabindex="-1" type="checkbox"/><span class="cbx x-refine__multi-select-cbx">Vaulted/Retired</span><span class="x-refine__multi-select-histogram"> (13,100)</span></a>
125
<a _sp="p2351460.m4118.c5" class="cbx x-ref

In [45]:
bs_links = bs.findAll('a')
bs_links[391]

<a _sp="p2351460.m4115.l8631" class="x-pagination__control" data-track='{"eventFamily":"LST","eventAction":"ACTN","actionKind":"NAVSRC","actionKinds":["NAVSRC"],"operationId":"2351460","flushImmediately":false,"eventProperty":{"moduledtl":"mi%3A4115%7Ciid%3A1%7Cli%3A1514%7Cluid%3Anext%7Ckind%3Apages%7C","pageci":"e05bb510-2199-11e9-adda-74dbd180a003","parentrq":"8b76b3ce1680aa66b659e893ffb540da"}}' href="https://www.ebay.com/sch/i.html?_from=R40&amp;_nkw=funko%2Bpop&amp;_sacat=0&amp;rt=nc&amp;LH_Sold=1&amp;LH_Complete=1&amp;_pgn=3" rel="next"><span aria-label="Go to next search page" role="img"><svg class="ebayui-image-clipped" xmlns="http://www.w3.org/2000/svg"><symbol id="ebayui-image-chevron-light-right" viewbox="0 0 24 32"><path d="M12.41 1.327a1.225 1.225 0 0 1 1.717 0l14.049 14.049c.234.234.39.546.39.859s-.156.624-.39.859L14.595 30.675c-.234.234-.546.39-.859.39s-.624-.156-.859-.39a1.225 1.225 0 0 1 0-1.717l12.722-12.722-13.19-13.19a1.225 1.225 0 0 1 0-1.717z"></path></symbol></sv

In [49]:
html = browser.html
bs = BeautifulSoup(html, 'html.parser')
bs_links = bs.findAll('a')
counter = 1
for bs_link in bs_links:
    if bs_link.text.find('3') != -1:
        print(counter)
        print(bs_link)
    counter = counter + 1

123
<a _sp="p2351460.m4118.c2" class="cbx x-refine__multi-select-link" data-track='{"eventFamily":"LST","eventAction":"ACTN","actionKind":"NAVSRC","actionKinds":["NAVSRC"],"operationId":"2351460","flushImmediately":false,"eventProperty":{"moduledtl":"mi%3A4118%7Ciid%3A1%7Cselected%3A0%7Cglobal%3A0%7CaspectURLParam%3ARarity%7Cli%3A8173%7Cluid%3A41%7C","pageci":"5ffb68c0-219b-11e9-8a91-74dbd18042ff","parentrq":"8b8086041680ab66cf90910affb35eee"}}' href="https://www.ebay.com/sch/i.html?_from=R40&amp;_nkw=funko%2Bpop&amp;_sacat=0&amp;LH_Sold=1&amp;LH_Complete=1&amp;rt=nc&amp;Rarity=Common&amp;_dcat=149372"><input aria-hidden="true" aria-label="Common" autocomplete="off" class="cbx x-refine__multi-select-checkbox " id="w_1548528683341_cbx" role="presentation" tabindex="-1" type="checkbox"/><span class="cbx x-refine__multi-select-cbx">Common</span><span class="x-refine__multi-select-histogram"> (13,399)</span></a>
124
<a _sp="p2351460.m4118.c3" class="cbx x-refine__multi-select-link" data-tr

In [51]:
html = browser.html
bs = BeautifulSoup(html, 'html.parser')
bs_links = bs.findAll('a')
counter = 1
target = 3
for bs_link in bs_links:
    if bs_link.text == str(target):
        print(counter)
        print(bs_link)
    counter = counter + 1

379
<a _sp="p2351460.m4115.l1513" aria-label="Page 3" data-track='{"eventFamily":"LST","eventAction":"ACTN","actionKind":"NAVSRC","actionKinds":["NAVSRC"],"operationId":"2351460","flushImmediately":false,"eventProperty":{"moduledtl":"mi%3A4115%7Ciid%3A1%7Cli%3A1514%7Cluid%3A2%7Ckind%3Apages%7C","pageci":"5ffb68c0-219b-11e9-8a91-74dbd18042ff","parentrq":"8b8086041680ab66cf90910affb35eee"}}' href="https://www.ebay.com/sch/i.html?_from=R40&amp;_nkw=funko%2Bpop&amp;_sacat=0&amp;rt=nc&amp;LH_Sold=1&amp;LH_Complete=1&amp;rt=nc&amp;_pgn=3">3</a>


In [ ]:
# <h1 class="srp-controls__count-heading">326,828 results</h1>

In [ ]:
# While the search results count - (page# * 50) > 50
#  Scrape listings on page
#  Add scrape to container
#  Find the (page# + 1) link
#  Click to advance the page
# Get the number of search results

In [39]:
browser.find_by_tag('a')[394].click()

'200<span class="clipped">Items Per Page</span>'

In [47]:
browser.find_by_tag('a')[391].click()

#### Function Based Scraping

10,index,Date,Rank,WeeklyGross,%Change,Theaters,Change,Avg.,Gross-to-Date,Week#
0,11,Feb 1622,1,"$291,954,422",-,4020,-,"$72,625","$291,954,422",1
1,12,Feb 23Mar 1,1,"$143,445,615",-50.9%,4020,-,"$35,683","$435,400,037",2
2,13,Mar 28,1,"$85,479,564",-40.4%,4084,+64,"$20,930","$520,879,601",3
3,14,Mar 915,1,"$57,496,927",-32.7%,3942,-142,"$14,586","$578,376,528",4
4,15,Mar 1622,1,"$35,881,708",-37.6%,3834,-108,"$9,359","$614,258,236",5
5,16,Mar 2329,2,"$25,178,398",-29.8%,3370,-464,"$7,471","$639,436,634",6
6,17,Mar 30Apr 5,3,"$17,489,106",-30.5%,2988,-382,"$5,853","$656,925,740",7
7,18,Apr 612,4,"$11,529,782",-34.1%,2747,-241,"$4,197","$668,455,522",8
8,19,Apr 1319,6,"$7,986,587",-30.7%,2180,-567,"$3,664","$676,442,109",9
9,20,Apr 2026,8,"$7,186,380",-10.0%,1930,-250,"$3,724","$683,628,489",10


10,index,Date,Rank,WeeklyGross,%Change,Theaters,Change,Avg.,Gross-to-Date,Week#
0,11,Dec 1521,1,"$296,602,356",-,4232,-,"$70,086","$296,602,356",1
1,12,Dec 2228,1,"$168,095,872",-43.3%,4232,-,"$39,720","$464,698,228",2
2,13,Dec 29Jan 4,2,"$84,264,374",-49.9%,4232,-,"$19,911","$548,962,602",3
3,14,Date(click to view chart),Rank,WeeklyGross,%Change,Theaters / Change,Avg.,Gross-to-Date,Week#,NaN
4,15,Jan 511,3,"$31,311,982",-62.8%,4232,-,"$7,399","$580,274,584",4
5,16,Jan 1218,5,"$17,443,892",-44.3%,3090,-1142,"$5,645","$597,718,476",5
6,17,Jan 1925,8,"$8,823,345",-49.4%,2456,-634,"$3,593","$606,541,821",6
7,18,Jan 26Feb 1,11,"$5,574,959",-36.8%,1745,-711,"$3,195","$612,116,780",7
8,19,Feb 28,14,"$3,319,499",-40.5%,1467,-278,"$2,263","$615,436,279",8
9,20,Feb 915,16,"$1,986,498",-40.2%,830,-637,"$2,393","$617,422,777",9


---
### Transformation
- Convert PDs to a csv format for ML application

In [16]:
# Determine how to reformat dates
dates = res2018['pds'][0]['Date']
dates
for date in dates:
    p_date = date.split('')[0].split(' ')
    if (len(p_date[1])==1):
        p_date[1] = '0'+ p_date[1]
    start_str = "-".join(p_date)+'-18'
    start_date = dt.strptime(start_str, '%b-%d-%y')
    week_nm = start_date.strftime('%U')
#     print(f"{start_str} = week # {week_nm}")

In [8]:
# function to convert weekend to calendar week number 
def getWkNum(datestring, yr):
    p_date = datestring.split('')[0].split(' ')
    if (len(p_date[1])==1):
        p_date[1] = '0'+ p_date[1]
    start_str = '-'.join(p_date) + f'-{yr}'
    start_date = dt.strptime(start_str, '%b-%d-%y')
    return start_date.strftime('%U')

getWkNum("Dec 30", 18)

'52'

In [9]:
# Function to convert gross strings into numbers
def stripUSD(amount):
    return ''.join(amount.replace('$', '').split(','))

,Year,BOM_id,Movie,W_00,W_01,W_02,W_03,W_04,W_05,W_06,W_07,W_08,W_09,W_10,W_11,W_12,W_13,W_14,W_15,W_16,W_17,W_18,W_19,W_20,W_21,W_22,W_23,W_24,W_25,W_26,W_27,W_28,W_29,W_30,W_31,W_32,W_33,W_34,W_35,W_36,W_37,W_38,W_39,W_40,W_41,W_42,W_43,W_44,W_45,W_46,W_47,W_48,W_49,W_50,W_51,W_52,T_00,T_01,T_02,T_03,T_04,T_05,T_06,T_07,T_08,T_09,T_10,T_11,T_12,T_13,T_14,T_15,T_16,T_17,T_18,T_19,T_20,T_21,T_22,T_23,T_24,T_25,T_26,T_27,T_28,T_29,T_30,T_31,T_32,T_33,T_34,T_35,T_36,T_37,T_38,T_39,T_40,T_41,T_42,T_43,T_44,T_45,T_46,T_47,T_48,T_49,T_50,T_51,T_52
0,2018,marvel2017b,Black Panther,0,0,0,0,0,0,291954422,143445615,85479564,57496927,35881708,25178398,17489106,11529782,7986587,7186380,6352126,4273996,2707174,1174457,746707,368118,217287,203114,113230,60009,42286,19869,45572,16592,90540,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,291954422,435400037,520879601,578376528,614258236,639436634,656925740,668455522,676442109,683628489,689980615,694254611,696961785,698136242,698882949,699251067,699468354,699671468,699784698,699844707,699886993,699906862,699952434,699969026,700059566,700059566,700059566,700059566,700059566,700059566,700059566,700059566,700059566,700059566,700059566,700059566,700059566,700059566,700059566,700059566,700059566,700059566,700059566,700059566,700059566,700059566,700059566
1,2018,theincredibles2,Incredibles 2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,269446690,124736824,81177900,44237078,26216948,19808208,12508642,8210310,5456248,3614118,2447947,5252592,1755520,955333,667224,353505,417506,276976,220915,180084,212891,166736,131124,66319,38406,25700,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,269446690,394183514,475361414,519598492,545815440,565623648,578132290,586342600,591798848,595412966,597860913,603113505,604869025,605824358,606491582,606845087,607262593,607539569,607760484,607940568,608153459,608320195,608451319,608517638,608556044,608581744,608581744,608581744,608581744,608581744
2,2018,foxmarvel18,Deadpool 2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,164707352,66620086,33688103,20865267,13019513,7999537,5971542,2584220,1191820,751414,440675,285867,153215,86806,50105,43040,10320,13518,3376,3239,1660,751,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,164707352,231327438,265015541,285880808,298900321,306899858,312871400,315455620,316647440,317398854,317839529,318125396,318278611,318365417,318415522,318458562,318468882,318482400,318485776,318489015,318490675,318491426,318491426,318491426,318491426,318491426,318491426,318491426,318491426,318491426,318491426,318491426,318491426,318491426
3,2018,missionimpossible6,Mission: Impossible - Fallout,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,89487371,52479913,28271437,15661939,11445869,10970238,5503570,3154330,1594266,863496,497901,228774,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,89487371,141967284,170238721,185900660,197346529,208316767,213820337,216974667,218568933,219432429,219930330,220159104,220159104,220159104,220159104,220159104,220159104,220159104,220159104,220159104,220159104,220159104,220159104,220159104
4,2018,untitledhansolostarwarsanthologyfilm,Solo: A Star Wars Story,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,119592692,41358782,22812613,14367783,6854245,4873982,1652879,661331,404452,257742,317944,207217,159987,103581,91639,32885,17758,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,119592692,160951474,183764087,198131870,204986115,209860097,211512976,212174307,212578759,212836501,213154445,213361662,213521649,213625230,213716869,213749754,213767512,213767512,213767512,213767512,213767512,213767512,213767512,213767512,213767512,213767512,213767512,213767512,213767512,213767512,213767512,213767512,213767512
5,2018,astarisborn2018,A Star is Born (2018),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,66160360,40915886,27501154,19957166,15475822,11480519,6496136,3930393,2671161,3549477,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [22]:
m2018cal_df.to_csv('Data/caly_mov_gross_2018.csv', index=False)

In [26]:
# Create column labels
wkl2018_cols = ['Year', 'BOM_id', 'Movie']
wkl_rows = []

for i in np.arange(52):
    lbl = i+1
    if lbl < 10:
        lbl = f'0{lbl}'
    wkl2018_cols.append(f'W_{lbl}')
for i in np.arange(52):
    lbl = i+1
    if lbl < 10:
        lbl = f'0{lbl}'
    wkl2018_cols.append(f'T_{lbl}')

# Look for the longest movie duration
max_dur = 0

# Make a row for each movie in the list
for i in np.arange(len(res2018['pds'])):
    # get the title and id to start the row
    new_row = ['2018', res2018['ids'][i], res2018['titles'][i]]
    # add in 0's 
    for j in np.arange(104):
        new_row.append('0')

    itr_pds = res2018['pds'][i]
    last_col = 56
    last_grs = 0
    row_count = itr_pds.shape[0]
    if (row_count > max_dur):
        max_dur = row_count
    
    for ind, row in itr_pds.iterrows():
        #wkNum = row['Week#']
        last_col = 55 + ind
        last_grs = stripUSD(row['Gross-to-Date'])
        new_row[3 + ind] = stripUSD(row['WeeklyGross'])
        new_row[last_col] = last_grs
    for j in np.arange(last_col+1, 107):
        new_row[j] = last_grs
    wkl_rows.append(new_row)

# Check the longest running movie
print(max_dur)

# Create dataframe out of columns and rows
m2018wkl_df = pd.DataFrame(wkl_rows, columns=wkl2018_cols)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(m2018wkl_df)

26


,Year,BOM_id,Movie,W_01,W_02,W_03,W_04,W_05,W_06,W_07,W_08,W_09,W_10,W_11,W_12,W_13,W_14,W_15,W_16,W_17,W_18,W_19,W_20,W_21,W_22,W_23,W_24,W_25,W_26,W_27,W_28,W_29,W_30,W_31,W_32,W_33,W_34,W_35,W_36,W_37,W_38,W_39,W_40,W_41,W_42,W_43,W_44,W_45,W_46,W_47,W_48,W_49,W_50,W_51,W_52,T_01,T_02,T_03,T_04,T_05,T_06,T_07,T_08,T_09,T_10,T_11,T_12,T_13,T_14,T_15,T_16,T_17,T_18,T_19,T_20,T_21,T_22,T_23,T_24,T_25,T_26,T_27,T_28,T_29,T_30,T_31,T_32,T_33,T_34,T_35,T_36,T_37,T_38,T_39,T_40,T_41,T_42,T_43,T_44,T_45,T_46,T_47,T_48,T_49,T_50,T_51,T_52
0,2018,marvel2017b,Black Panther,291954422,143445615,85479564,57496927,35881708,25178398,17489106,11529782,7986587,7186380,6352126,4273996,2707174,1174457,746707,368118,217287,203114,113230,60009,42286,19869,45572,16592,90540,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,291954422,435400037,520879601,578376528,614258236,639436634,656925740,668455522,676442109,683628489,689980615,694254611,696961785,698136242,698882949,699251067,699468354,699671468,699784698,699844707,699886993,699906862,699952434,699969026,700059566,700059566,700059566,700059566,700059566,700059566,700059566,700059566,700059566,700059566,700059566,700059566,700059566,700059566,700059566,700059566,700059566,700059566,700059566,700059566,700059566,700059566,700059566,700059566,700059566,700059566,700059566,700059566
1,2018,theincredibles2,Incredibles 2,269446690,124736824,81177900,44237078,26216948,19808208,12508642,8210310,5456248,3614118,2447947,5252592,1755520,955333,667224,353505,417506,276976,220915,180084,212891,166736,131124,66319,38406,25700,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,269446690,394183514,475361414,519598492,545815440,565623648,578132290,586342600,591798848,595412966,597860913,603113505,604869025,605824358,606491582,606845087,607262593,607539569,607760484,607940568,608153459,608320195,608451319,608517638,608556044,608581744,608581744,608581744,608581744,608581744,608581744,608581744,608581744,608581744,608581744,608581744,608581744,608581744,608581744,608581744,608581744,608581744,608581744,608581744,608581744,608581744,608581744,608581744,608581744,608581744,608581744,608581744
2,2018,foxmarvel18,Deadpool 2,164707352,66620086,33688103,20865267,13019513,7999537,5971542,2584220,1191820,751414,440675,285867,153215,86806,50105,43040,10320,13518,3376,3239,1660,751,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,164707352,231327438,265015541,285880808,298900321,306899858,312871400,315455620,316647440,317398854,317839529,318125396,318278611,318365417,318415522,318458562,318468882,318482400,318485776,318489015,318490675,318491426,318491426,318491426,318491426,318491426,318491426,318491426,318491426,318491426,318491426,318491426,318491426,318491426,318491426,318491426,318491426,318491426,318491426,318491426,318491426,318491426,318491426,318491426,318491426,318491426,318491426,318491426,318491426,318491426,318491426,318491426
3,2018,missionimpossible6,Mission: Impossible - Fallout,89487371,52479913,28271437,15661939,11445869,10970238,5503570,3154330,1594266,863496,497901,228774,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,89487371,141967284,170238721,185900660,197346529,208316767,213820337,216974667,218568933,219432429,219930330,220159104,220159104,220159104,220159104,220159104,220159104,220159104,220159104,220159104,220159104,220159104,220159104,220159104,220159104,220159104,220159104,220159104,220159104,220159104,220159104,220159104,220159104,220159104,220159104,220159104,220159104,220159104,220159104,220159104,220159104,220159104,220159104,220159104,220159104,220159104,220159104,220159104,220159104,220159104,220159104,220159104
4,2018,untitledhansolostarwarsanthologyfilm,Solo: A Star Wars Story,119592692,41358782,22812613,14367783,6854245,4873982,1652879,661331,404452,257742,317944,207217,159987,103581,91639,32885,17758,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,119592692,160951474,183764087,198131870,204986115,209860097,211512

In [29]:
m2018wkl_df.to_csv('Data/wkly_mov_gross_2018.csv', index=False)

#### Function Based Transformation:

In [15]:
def transform_movie(scrape_res, affix, year):
#     cal_cols = ['Year', 'BOM_id', 'Movie']
#     cal_rows = []
    
#     for i in np.arange(53):
#         lbl = i
#         if i < 10:
#             lbl = f'0{i}'
#         cal_cols.append(f'W_{lbl}')
    
#     for i in np.arange(53):
#         lbl = i
#         if i < 10:
#             lbl = f'0{i}'
#         cal_cols.append(f'T_{lbl}')
        
#     # for each movie in the list
#     for i in np.arange(len(scrape_res['pds'])):
#         # get the title and id to start the row
#         new_row = [year, scrape_res['ids'][i], scrape_res['titles'][i]]
#         # add in 0's 
#         for j in np.arange(106):
#             new_row.append('0')

#         itr_pds = scrape_res['pds'][i]
#         last_col = 56
#         last_grs = 0
#         for ind, row in itr_pds.iterrows():
#             calWkNum = getWkNum(row['Date'], year)
#             last_col = 56 + int(calWkNum)
#             last_grs = stripUSD(row['Gross-to-Date'])
#             new_row[3 + int(calWkNum)] = stripUSD(row['WeeklyGross'])
#             new_row[last_col] = last_grs
#         for j in np.arange(last_col+1, 109):
#             new_row[j] = last_grs
#         cal_rows.append(new_row)
    
#     mov_cal_df = pd.DataFrame(cal_rows, columns=cal_cols)
#     save_path = f'Data/caly_mov_gross_{affix}.csv'
#     mov_cal_df.to_csv(save_path, index=False)
    
    # Create column labels
    wkl_cols = ['Year', 'BOM_id', 'Movie']
    wkl_rows = []

    for i in np.arange(52):
        lbl = i+1
        if lbl < 10:
            lbl = f'0{lbl}'
        wkl_cols.append(f'W_{lbl}')
    for i in np.arange(52):
        lbl = i+1
        if lbl < 10:
            lbl = f'0{lbl}'
        wkl_cols.append(f'T_{lbl}')

    # Look for the longest movie duration
    max_dur = 0

    # Make a row for each movie in the list
    for i in np.arange(len(scrape_res['pds'])):
        # get the title and id to start the row
        new_row = [year, scrape_res['ids'][i], scrape_res['titles'][i]]
        # add in 0's 
        for j in np.arange(104):
            new_row.append('0')

        itr_pds = scrape_res['pds'][i]
        last_col = 56
        last_grs = 0
        row_count = itr_pds.shape[0]
        if (row_count > max_dur):
            max_dur = row_count

        for ind, row in itr_pds.iterrows():
            #wkNum = row['Week#']
            last_col = 55 + ind
            last_grs = stripUSD(row['Gross-to-Date'])
            new_row[3 + ind] = stripUSD(row['WeeklyGross'])
            new_row[last_col] = last_grs
        for j in np.arange(last_col+1, 107):
            new_row[j] = last_grs
        wkl_rows.append(new_row)
        
    mov_wkl_df = pd.DataFrame(wkl_rows, columns=wkl_cols)
    save_path = f'Data/wkly_mov_gross_{affix}.csv'
    mov_wkl_df.to_csv(save_path, index=False)

In [16]:
transform_movie(res2017, "2017", 17)
transform_movie(res2016, "2016", 16)

KeyError: 'Gross-to-Date'